# Llama 3.2 한글 Test

Test Model

In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [2]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.0 MB/s eta 0:00:00


In [3]:
!pip install -q accelerate>=0.26.0

In [4]:
# The model that you want to train from the Hugging Face hub
base_model = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# Base Model

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer
import torch

In [6]:

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Set pad_token_id to avoid receiving warning messages.

In [7]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [8]:
def generate_response(messages, model):

    input_ids = tokenizer.apply_chat_template(
         messages, add_generation_prompt=True,
         return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    decoded_output = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    return decoded_output

instruction = "BTS 음악의 컨셉은 무엇입니까?"
messages = [
    {"role": "user", "content": f"{instruction}"},
]

generate_response(messages, model)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'BTS는 한국의 K-pop 그룹으로, 다양한 음악 컨셉을 통해 다양한 메시지를 전달하고 있습니다. 그들의 음악 컨셉은 시간이 지남에 따라 변동되지만, 주요 특징을 몇 가지 정리해 보겠습니다.\n\n1. **사랑과 인생**: BTS의 초기 앨범에서부터 사랑과 인생에 대한 메시지를 전달하는 것이 주요 컨셉 중 하나였습니다. 예를 들어, "No More Dream", "We Are Bulletproof Pt.2" 등의 곡에서 사랑과 인생의 중요성을 강조했습니다.\n\n2. **정신 건강**: "Map of the Soul: Persona" 앨범에서는 정신 건강과 스트레스에 대한 메시지를 전달했습니다. 이 앨범에서는 "Agust D"와 "My Time" 등의 곡에서 정신 건강에 대한 중요성을 강조했습니다.\n\n3. **사회적 문제**: BTS는 사회적 문제에 대한 관심을 보이며, "Love Myself" 앨범에서는 개인의 자아를 찾고, "Map of the Soul: 7" 앨범에서는 사회적 문제에 대한 메시지를 전달했습니다. 예를 들어, "Euphoria"와 "Black Swan" 등의 곡에서는 사회적 문제를 다루고 있습니다.\n\n4. **자아 발견**: BTS는 자아 발견과 개인의 성장을 다루는 곡도 많이 있습니다. "Map of the Soul: 7" 앨범에서는 "Dionysus"와 "On" 등의 곡에서 자아 발견과 성장을 다루고 있습니다.\n\n5. **다양한 문화**: BTS는 다양한 문화를 다루는 곡도 많이 있습니다. 예를 들어, "Love Myself" 앨범에서는 "Singularity"와 "Outro: Love Myself" 등의 곡에서 다양한 문화를 다루고 있습니다.\n\nBTS의 음악 컨셉은 시간이 지남에 따라 변동되며, 그들의 음악은 다양한 메시지를 전달하는 데 중요한 역할을 합니다.'

Create the text generation pipeline with model and tokenizer.

In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Device set to use cuda:0


Run the pipeline with the custom prompt

In [10]:

messages = [{"role": "user", "content": "BTS 음악의 컨셉은 무엇입니까?"}]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=terminators, do_sample=True)

print(outputs[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 06 Dec 2024

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

BTS 음악의 컨셉은 무엇입니까?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

BTS(방탄소년단)는 K-pop 그룹으로, 음악, 비디오, 미디어, 그리고 문화에서 다양한 컨셉을 사용하고 있습니다. 그들의 컨셉은 주로 다음과 같은 주제를 다루고 있습니다:

1. **자아발견과 성장**: BTS는 자신의 정체성과 자아발견을 주제로 자주 다루고 있습니다. 그들은 자신의 경력과 음악에 대해 자주 이야기하며, 그들의 음악이 가지는 의미를 설명합니다. 예를 들어, "DNA"는 자신이 자신의 정체성을 자주 강조하며, "Euphoria"는 자신들의 음악이 가지는 열정과 감정을 표현합니다.

2. **사회적 문제와 사회적 문제**: BTS는 다양한 사회적 문제를 주제로 다루며, 그들의 음악을 통해 문제를 인식하고, 해결 방안을 제시합니다. "Fake Love"는 불공정한 사회적 문제를 다루며, "Idol"은 자신들의 직업에 대한 고민을 표현합니다.

3. ** tình애와 사랑**: BTS는 또한 사랑과 관련된 주제를 다루며, "Boy With Luv"는 사랑과 friendship를 주제로, "Mikrokosmos"는 가족과 가족의 사랑을 표현합니다.

4. **성장과 발전**: 그들의 음악은 또한 성장과 발전을 주제로 다루며, "Spring Day"는 그들의 성장 과정을 반영하고, "Promise"

In [15]:
messages2 = [{"role": "user", "content": "BTS 음악의 컨셉을 3개만 적어주세요."}]

prompt2 = tokenizer.apply_chat_template(
    messages2, tokenize=False, add_generation_prompt=True
)

outputs2 = pipe(prompt2, max_new_tokens=1024, eos_token_id=terminators, do_sample=True)

print(outputs2[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 06 Dec 2024

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

BTS 음악의 컨셉을 3개만 적어주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

BTS는 다양한 음악 컨셉을 통해 다양한 장르와 메시지를 전달하고 있습니다. 몇 가지 주요 컨셉을 살펴보겠습니다:

1. **자아 발전과 성장**: BTS의 많은 곡들은 개인의 자아 발전과 성장에 대한 이야기를 다룹니다. 예를 들어, "No More Dream"는 꿈을 이루기 위한 희망과 노력의 중요성을 강조하며, "Boy With Luv"는 사랑과 동반자의 중요성을 다룹니다.

2. **사회적 문제와 사회적 책임**: BTS는 다양한 사회적 문제에 대해 음악을 통해 주목받고 있습니다. "DNA"는 지속 가능한 지구를 지키는 중요성을 강조하며, "Spring Day"는 전쟁과 피해자에 대한 애도와 희생을 다룹니다.

3. **사랑과 인생의 의미**: BTS는 사랑과 인생의 의미에 대해 다양한 방식으로 표현합니다. "Euphoria"는 사랑과 감정의 깊이, "Love Yourself"는 사랑의 중요성을 강조하며, "Not Today"는 사랑과 인생의 의미에 대한 깊은 생각을 담고 있습니다.

이 외에도 BTS는 다양한 컨셉을 통해 음악을 전달하며, 다양한 장르와 메시지를 전달하고 있습니다.
